In [1]:
from numba import cuda
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)


cuda.select_device(0)
cuda.close()
cuda.select_device(0)

torch.cuda.empty_cache()


df = pd.read_csv("15_new_DS.csv")
df.head()


,path,sentence
0,/home/peterr/macocu/task6_speech/data/00000001.flac.wav,poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice
1,/home/peterr/macocu/task6_speech/data/00000002.flac.wav,prvi klub koji se javio za stanku je klub esdepea
2,/home/peterr/macocu/task6_speech/data/00000003.flac.wav,u njihovo ime predstavnik bojan glavašević
3,/home/peterr/macocu/task6_speech/data/00000004.flac.wav,hvala gospodine predsjedniče
4,/home/peterr/macocu/task6_speech/data/00000005a.flac.wav,poštovani gospodine predsjedniče hrvatskog sabora uvažene kolegice i kolege klub zastupnika esdepea


In [2]:
LIM = int(df.shape[0] * 0.8)
common_voice_test_df = df.loc[LIM:, :].copy().reset_index(drop=True)

In [3]:

import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

#common_voice_train_df.loc[:, "audio"] = common_voice_train_df.path.apply(load_audio)
common_voice_test_df.loc[:, "audio"] = common_voice_test_df.path.apply(load_audio)

#common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)





from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


#common_voice_train_mapped = common_voice_train_dataset.map(
    #prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/7464 [00:00<?, ?ex/s]

In [4]:
repo_name = "15_/checkpoint-13600"
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)#.to("cuda")

rezs = list()
for i in common_voice_test_df.index:
    try:
        input_dict = processor(common_voice_test_mapped[i]["input_values"], return_tensors="pt", padding=True,
        sampling_rate=16000)
        #logits = model(input_dict.input_values.to("cuda")).logits
        logits = model(input_dict.input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]

        gold = common_voice_test_df.sentence[i]   
        prediction = processor.decode(pred_ids)
        fullpath = common_voice_test_df.path[i]
        filename = fullpath.split("/")[-1]
        rezs.append(prediction)
    except Exception as e:
        logging.error(f"Got exception: {e} for {common_voice_test_df.loc[i, 'path']}")
        break
df[repo_name] = rezs

ValueError: Length of values does not match length of index

In [5]:
common_voice_test_df[repo_name] = rezs

In [6]:
repo_name = "classla/wav2vec2-xls-r-sabor-hr"
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)#.to("cuda")

rezs = list()
for i in common_voice_test_df.index:
    try:
        input_dict = processor(common_voice_test_mapped[i]["input_values"], return_tensors="pt", padding=True,
        sampling_rate=16000)
        #logits = model(input_dict.input_values.to("cuda")).logits
        logits = model(input_dict.input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]

        gold = common_voice_test_df.sentence[i]   
        prediction = processor.decode(pred_ids)
        fullpath = common_voice_test_df.path[i]
        filename = fullpath.split("/")[-1]
        rezs.append(prediction)
    except Exception as e:
        logging.error(f"Got exception: {e} for {common_voice_test_df.loc[i, 'path']}")
        break
common_voice_test_df[repo_name] = rezs

2021-12-21 20:04:34,829 - INFO - Lock 140543479296736 acquired on /home/peterr/.cache/huggingface/transformers/c0ea789db14b10c3770ca3a4663bdaa38015450af8994b5f118e5468b12afd76.11085e4eec2577ef9e8dd777850165ff20ab2f2cc53ffc9d3eacb0ace3499e5d.lock


Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

2021-12-21 20:04:35,337 - INFO - Lock 140543479296736 released on /home/peterr/.cache/huggingface/transformers/c0ea789db14b10c3770ca3a4663bdaa38015450af8994b5f118e5468b12afd76.11085e4eec2577ef9e8dd777850165ff20ab2f2cc53ffc9d3eacb0ace3499e5d.lock
2021-12-21 20:04:35,838 - INFO - Lock 140521297973056 acquired on /home/peterr/.cache/huggingface/transformers/880bda4289edd96bf44b61b003ffc2b899903d4918473659f16a3ab984bc967c.7583a94325f6aa462d5b0168d6360991f0e661b41399e08ff90d66ca8f04687b.lock


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

2021-12-21 20:05:03,961 - INFO - Lock 140521297973056 released on /home/peterr/.cache/huggingface/transformers/880bda4289edd96bf44b61b003ffc2b899903d4918473659f16a3ab984bc967c.7583a94325f6aa462d5b0168d6360991f0e661b41399e08ff90d66ca8f04687b.lock


In [13]:
common_voice_test_df.drop(columns=["audio"]).to_csv("15_comparison_of_models.csv", index=False)

In [15]:
common_voice_test_df.drop(columns=["audio"]).head()

,path,sentence,15_/checkpoint-13600,classla/wav2vec2-xls-r-sabor-hr
0,/home/peterr/macocu/task6_speech/data/00018794.flac.wav,dakle što je cilj odnosno što je prava svrha ovog zakona,dakle što je cilj odnosno što je prava svrha ovog zakona,dakle što je cilj odnosno što je prava svrha ovog zakona
1,/home/peterr/macocu/task6_speech/data/00018795a.flac.wav,cilj je određenom metodologijom razviti infrastrukturu,cil je određenom metodologijom razvij infrastruktur,cil je odredeno metodologijom razvit inrastrukturu
2,/home/peterr/macocu/task6_speech/data/00018795b.flac.wav,infrastrukturu potrebno da bi se uopće kroz korištenje razno raznih poticajnih mjera,infrastrukturu potrebno da bi se uopće kroz korištenje raznoraznih poticajnih mjera,inrastrukturu potrebno da bi se uopće kroz korištenje raznoraznih poticajinih mjera
3,/home/peterr/macocu/task6_speech/data/00018795c.flac.wav,na tržište erha stavilo vozila u izvjesnoj količini a koja onda ne bi bila limitirana,na tržište erha stavilo vozila u izvjesnoj količini a koja onda ne bi bila limitirana,na tržište erha stavilo vozila u izvisnoj količini a koja onda ne bi bila limitirana
4,/home/peterr/macocu/task6_speech/data/00018795d.flac.wav,radijusom kretanja i nekim drugim limitima već bi kroz predmetnu infrastrukturu,radio samkretanja i nekim drugim limitima već bi kroz predmetnu infrastrukturu,radiu sam kretanja i nekim drugim limitima većbi kroz predmetnu inrastrukturu
